# Real City Road Network Speed Information Generator
This notebook gives a brief overview on how to add speed information.

In [1]:
DATA_FOLDER = "../data_example/"

I used Seoul open data set, but you can use your own implementation.

Load data which is downloaded from https://data.seoul.go.kr/. Note that it includes lots of Korean words.

In [2]:
import pandas as pd
data = pd.read_csv(DATA_FOLDER+'road_speed_seoul_20181023.csv', encoding='euc-kr')

Load graph data.

In [3]:
import osmnx as ox
G = ox.load_graphml(filepath=DATA_FOLDER+'seoul_rectangular_drive_network_simplified.graphml')

Following code matches road name in OSMNX and road name in speed data.
Actual speed information is not included. Only road name in speed data would be exported.

In [4]:
# 도로명 means road name
road_name_list = list(set(list(data['도로명'])))

(1) Initialize road speed if name matches

In [5]:
from queue import Queue
q = Queue()
match_road_name_count = 0
match_road_name_set = set()
for edge in list(G.edges(keys=True, data=True)):
    u, v, k, info = edge
    road_name = info.get('name')
    # name matches
    if road_name in road_name_list:
        match_road_name_count += 1
        match_road_name_set.add(road_name)
        info['speed_info_closest_road'] = road_name
        q.put(edge)
print("Match road name count: ", match_road_name_count)
print("Unique road name count: ", len(match_road_name_set))

Match road name count:  5913
Unique road name count:  424


(2) If name does not match, set to the closest speed-known road.

In [6]:
while q.qsize() > 0:
    p = q.get()
    u0, v0, k0, info0 = p
    for e in G.out_edges(v0, keys=True, data=True):
        u, v, k, info = e
        if 'speed_info_closest_road' not in info:
            info['speed_info_closest_road'] = info0['speed_info_closest_road']
            q.put(e)

Finally export data that contains road name for speed data.

In [7]:
ox.save_graphml(G, filepath=DATA_FOLDER+'seoul_rectangular_drive_network_simplified_with_speed.graphml')